In [ ]:
!pip install -q sentence-transformers faiss-cpu llama-cpp-python langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 MB 18.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA

In [ ]:
# Loading the file
loader = TextLoader("/content/drive/MyDrive/DE/DL/sample.txt")
document = loader.load()

# Splitting into chunks
splitter = CharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
docs = splitter.split_documents(document)

# Embed and index
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

# Storing in vectorDB
vectorDB = FAISS.from_documents(docs, embeddings)

In [ ]:
# Directly taking a model via link using wget
!wget -q https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf -O tinyllama.gguf

In [ ]:
# LLM Model
llm = LlamaCpp(
    model_path = "tinyllama.gguf",
    temperature = 0.5,
    max_tokens = 512,
    top_p = 0.95,        # 95% meaningful sentences
    n_ctx = 2048,        # Reg. tokens
    verbose = False      # If true, process in backend paaka mudiyum
)

# LangChain
qa_chain = RetrievalQA.from_chain_type(llm = llm, retriever = vectorDB.as_retriever())

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


In [ ]:
question = "Who is Virat Kohli?"
answer = qa_chain.run(question)

print("Answer:\n", answer)

Answer:
  Virat Kohli is an Indian cricket captain, born on November 5, 1988. He is one of the most celebrated and consistent cricketers in the world.
